In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# define filepaths
products_path = "./data/products_train.csv"
train_sessions_path = "./data/sessions_train.csv"
test_sessions_path = "./data/sessions_test_task1.csv"

# load into dataframes
products_df = pd.read_csv(products_path)
train_whole_df = pd.read_csv(train_sessions_path)
test_df = pd.read_csv(test_sessions_path)

# convert previos items strings to lists
train_whole_df['prev_items'] = train_whole_df['prev_items'].str.strip("[']").str.replace("\n", "").str.split("' '")
test_df['prev_items'] = test_df['prev_items'].str.strip("[']").str.replace("\n", "").str.split("' '")

# create seperate train and validation sets
# this enables testing the models before making a submition to the competition
train_df, val_df = train_test_split(train_whole_df, test_size=0.2)

#display(products_df)
display(train_df)
display(val_df)
display(test_df)

,prev_items,next_item,locale
1873641,"[B09NDH6WXG, B09NDH6WXG, B09NDH6WXG]",B00264J1OY,JP
3315058,"[B07H3Z3LMM, B07H3XCW4L]",B01EN1FYI4,ES
1767352,"[B08CR5D69W, B07P1QC89M, B07P53HDSP, B07HFRDK3V]",B07WP4NGLD,JP
2969581,"[B0006TXE30, B0B14X5PZ8]",B09TT9MH33,UK
182875,"[B084CJ395S, B00QXIXV3Q, B07HSCHNTK, B0B8ND6J7...",B08XMRNJ57,DE
...,...,...,...
130468,"[B08HYXSZJL, B08HYXTLDC, B0BFPK954P, B08Y1S26Y...",B08TWDSXDC,DE
3163072,"[B07TQH3TSP, B07TRG2Y58, B07TN8SXSP]",B07TRG2RGH,UK
2577224,"[B0861G32ZP, B08FJ3Y7LJ, B0861G32ZP]",B01N8RII00,UK
3200240,"[B001P5UAV8, B001P5UAV8, B07SQ52CY4]",B0784XN6Q4,UK


,prev_items,next_item,locale
1451111,"[B07XG2RLGK, B07JDRLGY3, B07GV8X19Q, B07GV78JL...",B09QSYTD4G,JP
3046383,"[B00KAB6KXK, B09KNZD5MR, B07F4RBQQQ, B07FBX5RY...",B07FJQTN69,UK
2022807,"[B07P8S8SQM, B07TBG55H6]",B0824QHFWY,JP
195992,"[B07QKY52XL, B07P6KJC5C, B015FO9XF8, B01AK6RKCO]",B014SPNQV2,DE
875349,"[B08NW1SDNS, B08NW1SDNS, B07MZ4K67Y, B07MZ4K67Y]",B09R1TMXR3,DE
...,...,...,...
1564904,"[B07ZSHKBT2, B00FWIDVVU, B00CJRFFH0]",B08VXG9HWW,JP
586188,"[B0899H63GK, B089999MFS, B01M6C36H1, B085J193F...",B08WDBFJTS,DE
466237,"[B01MYBT40B, B01NAWSIL2, B01MSB0C7U, B01NBYCNF...",B01NCZNEHD,DE
1530024,"[B00F7IDYN0, B01LXIPL3L, B01LXIPL3L]",B0BB6QK63L,JP


,prev_items,locale
0,"[B08V12CT4C, B08V1KXBQD, B01BVG1XJS, B09VC5PKN...",DE
1,"[B00R9R5ND6, B00R9RZ9ZS, B00R9RZ9ZS]",DE
2,"[B07YSRXJD3, B07G7Q5N6G, B08C9Q7QVK, B07G7Q5N6G]",DE
3,"[B08KQBYV43, 3955350843, 3955350843, 395535086...",DE
4,"[B09FPTCWMC, B09FPTQP68, B08HMRY8NG, B08TBBQ4B...",DE
...,...,...
316966,"[B077SZ2C3Y, B0B14M3VZX]",UK
316967,"[B08KFHDPY9, B0851KTSRZ, B08KFHDPY9, B0851KTSR...",UK
316968,"[B07PY1N81F, B07Q1Z8SQN, B07PY1N81F, B07Q1Z8SQ...",UK
316969,"[B01MCQMORK, B09JYZ325W]",UK


In [19]:
test_sessions = pd.read_csv("data/sessions_test_task1.csv")
test_sessions.head()

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE


In [20]:
def stupid_predictions(locale, sess_test_locale):
    most_common = list(train_whole_df[train_whole_df["locale"] == locale]["next_item"].value_counts()[:100].index)
    predictions = []
    for _ in range(len(sess_test_locale)):
        predictions.append(
            most_common
        ) 
    sess_test_locale['next_item_prediction'] = predictions
    sess_test_locale.drop('prev_items', inplace=True, axis=1)
    return sess_test_locale

In [21]:
predictions = []
test_locale_names = test_sessions['locale'].unique()
for locale in test_locale_names:
    sess_test_locale = test_sessions.query(f'locale == "{locale}"').copy()
    predictions.append(
        stupid_predictions(locale, sess_test_locale)
    )
predictions = pd.concat(predictions).reset_index(drop=True)
predictions.sample(5)

,locale,next_item_prediction
39477,DE,"[B000T9G3X0, B0BGC82WVW, B09XBG1T3K, B0BF7CHRW..."
223683,UK,"[B08FZFDM7Z, B07KHPBF28, B0B7B75TVW, B0B5V31S6..."
315471,UK,"[B08FZFDM7Z, B07KHPBF28, B0B7B75TVW, B0B5V31S6..."
164069,JP,"[B086WCH4G1, B07Z7YH98P, B07KRJMTXH, B07X6WLV9..."
240023,UK,"[B08FZFDM7Z, B07KHPBF28, B0B7B75TVW, B0B5V31S6..."


In [26]:
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = products_df.query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"

In [27]:
check_predictions(predictions)

In [29]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(f'data/submissions/stupid_submission_task1.parquet', engine='pyarrow')

In [ ]:
!aicrowd login

In [ ]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f "submission_task1.parquet"